In [ ]:
# Constructing a Growth Model

In [12]:
import numpy as np

# State index mapping
# 0: New
# 1: Paying Current
# 2: Non-Paying Current
# 3: Reactivated
# 4: Resurrected
# 5: At Risk WAU
# 6: At Risk MAU
# 7: Dormant

# Transition matrix
# Each row represents transitions from one state to other states.
transition_matrix = np.array([
    [0, 0.1, 0.1, 0, 0, 0.8, 0, 0],  # New -> Paying Current, Non-Paying Current, At Risk WAU
    [0, 0.6, 0, 0, 0, 0.4, 0, 0],  # Paying Current -> Paying Current, At Risk WAU
    [0, 0.07, 0.53, 0, 0, 0.4, 0, 0],  # Non-Paying Current -> Paying Current, Non-Paying Current, At Risk WAU
    [0, 0.25, 0.25, 0, 0, 0, 0.5, 0],  # Reactivated -> Paying Current, Non-Paying Current, At Risk MAU
    [0, 0.25, 0.25, 0, 0, 0, 0.5, 0],  # Resurrected -> Paying Current, Non-Paying Current, At Risk MAU
    [0, 0, 0, 0, 0, 0, 1, 0],  # At Risk WAU -> At Risk MAU
    [0, 0, 0, 0, 0.2, 0, 0, 0.8],  # At Risk MAU -> Resurrected, Dormant
    [0, 0, 0, 0, 0.2, 0, 0, 0.8]  # Dormant -> Resurrected, Dormant
])

In [13]:
# Starting at New
state = np.array([1, 0, 0, 0, 0, 0, 0, 0])

In [23]:
def simulate(users, transition_matrix, days=100):
    for _ in range(days):
        users = users.dot(transition_matrix)
    return users

# Initial number of users
users = np.array([1000000, 0, 0, 0, 0, 0, 0, 0])

# Baseline simulation
baseline_users = simulate(users, transition_matrix)
baseline_paying = baseline_users[1]

# Sensitivity analysis
delta = 0.01
max_change = 0
max_change_transition = None

for i in range(transition_matrix.shape[0]):
    for j in range(transition_matrix.shape[1]):
        # Skip if increasing transition probability would make it > 1
        # or if transition is from state 7 to state 1
        if transition_matrix[i][j] + delta > 1 or (i == 7 and j == 1):
            continue
        
        # Temporarily increase transition probability
        transition_matrix[i][j] += delta

        # New simulation
        new_users = simulate(users, transition_matrix)
        new_paying = new_users[1]

        # Calculate change in paying customers
        change = new_paying - baseline_paying

        # Check if this is the biggest change we've seen
        if abs(change) > abs(max_change):
            max_change = change
            max_change_transition = (i, j)

        # Reset transition probability
        transition_matrix[i][j] -= delta


print(f"The transition with the biggest impact is from state {max_change_transition[0]} to state {max_change_transition[1]}, with a change of {max_change} paying customers.")

The transition with the biggest impact is from state 7 to state 3, with a change of 61024.93513322051 paying customers.


In [ ]:
state_names = {
    0: 'New',
    1: 'Paying Current',
    2: 'Non-Paying Current',
    3: 'Reactivated',
    4: 'Resurrected',
    5: 'At Risk WAU',
    6: 'At Risk MAU',
    7: 'Dormant'
}